In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
class SingleLayerNetwork:
    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()

    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
        
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]),
                             name='weights')        
            b1 = tf.Variable(tf.zeros([num_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1, name='hidden1')
        
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]), name='weights')
            b0 = tf.Variable(tf.zeros([10]), name='biases')
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name='softmax')

        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)

        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                              tf.float32), name='accuracy')

        tf.scalar_summary("loss", loss)
        tf.scalar_summary("accuracy", accuracy)
        tf.histogram_summary("weights_hidden", w1)
        tf.histogram_summary("biases_hidden", b1)
        tf.histogram_summary("weights_output", w0)
        tf.histogram_summary("biases_output", b0)
                
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
            
    def prepare_session(self):
        sess = tf.InteractiveSession()
        sess.run(tf.initialize_all_variables())
        summary = tf.merge_all_summaries()
        writer = tf.train.SummaryWriter("/tmp/mnist_sl_logs", sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

In [4]:
!rm -rf /tmp/mnist_sl_logs

In [5]:
nn = SingleLayerNetwork(1024)

i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x: batch_xs, nn.t: batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run(
            [nn.summary, nn.loss, nn.accuracy],
            feed_dict={nn.x:mnist.test.images, nn.t: mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)

Step: 100, Loss: 3193.628174, Accuracy: 0.903000
Step: 200, Loss: 2361.745117, Accuracy: 0.929700
Step: 300, Loss: 1984.334229, Accuracy: 0.941500
Step: 400, Loss: 2004.437256, Accuracy: 0.938000
Step: 500, Loss: 1718.406616, Accuracy: 0.948100
Step: 600, Loss: 1502.005127, Accuracy: 0.954600
Step: 700, Loss: 1291.457520, Accuracy: 0.961200
Step: 800, Loss: 1256.212280, Accuracy: 0.961100
Step: 900, Loss: 1196.300659, Accuracy: 0.964400
Step: 1000, Loss: 1259.283325, Accuracy: 0.964000
Step: 1100, Loss: 1083.064819, Accuracy: 0.966200
Step: 1200, Loss: 1062.172607, Accuracy: 0.966900
Step: 1300, Loss: 1196.362305, Accuracy: 0.962300
Step: 1400, Loss: 1086.707153, Accuracy: 0.967000
Step: 1500, Loss: 1111.764526, Accuracy: 0.967500
Step: 1600, Loss: 1120.840698, Accuracy: 0.966900
Step: 1700, Loss: 978.348206, Accuracy: 0.970500
Step: 1800, Loss: 978.814453, Accuracy: 0.970800
Step: 1900, Loss: 997.710693, Accuracy: 0.970500
Step: 2000, Loss: 1000.483398, Accuracy: 0.971600
